Set these two paths

In [17]:
results_path = '/home/mjw/OneDrive/Microgrid MG2Lab/Dataset/JPL_v4/Forecast/Seasonal offline test 2/'
mg2lab_dataset_path = '/home/mjw/OneDrive/Microgrid MG2Lab/Dataset/'

Pandas and some error calcs

In [18]:
import pandas as pd

pd.options.plotting.backend='plotly'

def calc_mae(y:pd.Series,yhat:pd.Series,norm=1):
    return (y-yhat).abs().mean()/norm

def calc_rmse(y:pd.Series,yhat:pd.Series,norm=1):
    return (((y-yhat)**2).mean())**0.5/norm

def calc_skill(true:pd.Series,forecast:pd.Series,benchmark:pd.Series,method='mae',norm=1):
    if method=='mae':
        return 1 - calc_mae(true,forecast,norm)/calc_mae(true,benchmark,norm)
    if method=='rmse':
        return 1 - calc_rmse(true,forecast,norm)/calc_rmse(true,benchmark,norm)

Arrange seasonal data together

In [19]:
seasons = ['winter','spring','summer','autumn']

#df = pd.read_csv('../../test_offline_seasonal_JPL_v4.csv',index_col=0,parse_dates=True,comment='#')
df = pd.read_csv(mg2lab_dataset_path+'JPL_v4/'+'all_JPL_v4.csv',index_col=0,parse_dates=True,comment='#')

df['persist'] = df.power.shift(96*7).values

#df.columns = ['LoadKW']

load = {}
load['winter'] = df['2018-12-3':'2018-12-16']
load['spring'] = df['2019-3-18':'2019-3-31']
load['summer'] = df['2019-8-5':'2019-8-18']
load['autumn'] = df['2019-10-14':'2019-10-27']
    
forecast = {}
for season in seasons:
    forecast[season] = pd.read_csv(f'{results_path}test_offline_{season}_v2.csv',index_col=1,parse_dates=True,comment='#')
    forecast[season].timestamp_utc = pd.to_datetime(forecast[season].timestamp_utc)
    
for season in seasons:
    load[season].index = pd.date_range(forecast[season].index[0],periods=len(load[season]),freq='15min')


Calculate and plot daily errors

In [20]:
week=1 # 0 or 1
update_h = 24 # 24 or 1

t_range = pd.date_range('2023-1-9',periods=7*int(24/update_h),freq=f'{update_h}h') + pd.Timedelta(days=week*7)
winter = pd.DataFrame()
for t in t_range:
    times_next_day = pd.date_range(t,periods=4*update_h,freq='15min')
    forecast_next_day = forecast['winter'].loc[t:t+pd.Timedelta('1d')][:4*update_h].predicted_activepower_ev_1.values
    next_day = pd.DataFrame({'Forecast kW':forecast_next_day},
                            index=times_next_day)
    winter = pd.concat([winter,next_day])

winter['Load kW'] = load['winter']['power']
winter['Persist 7d kW'] = load['winter']['persist']
winter.index = pd.date_range('2018-12-3',periods=7*96,freq='15min') + pd.Timedelta(days=week*7)

err = {'Forecast nMAE %':[],'Forecast nRMSE %':[],'Persist nMAE %':[],'Persist nRMSE %':[],'Skill nMAE %':[],'Skill nRMSE %':[]}
err_index = []
for day in winter.index.day.unique():
    winter_day = winter.loc[winter.index.day==day]
    err_index.append(winter_day.index[0])
    err['Forecast nMAE %'].append(100*calc_mae(winter_day['Load kW'],winter_day['Forecast kW'],norm=25))
    err['Forecast nRMSE %'].append(100*calc_rmse(winter_day['Load kW'],winter_day['Forecast kW'],norm=25))
    err['Persist nMAE %'].append(100*calc_mae(winter_day['Load kW'],winter_day['Persist 7d kW'],norm=25))
    err['Persist nRMSE %'].append(100*calc_rmse(winter_day['Load kW'],winter_day['Persist 7d kW'],norm=25))
    err['Skill nMAE %'].append(100*calc_skill(winter_day['Load kW'],winter_day['Forecast kW'],winter_day['Persist 7d kW'],method='mae',norm=25))
    err['Skill nRMSE %'].append(100*calc_skill(winter_day['Load kW'],winter_day['Forecast kW'],winter_day['Persist 7d kW'],method='rmse',norm=25))

err = pd.DataFrame(err,index=err_index)

for col in err.columns:
    err[col] = err[col].round(1)
    
f = winter.plot(title=f'Day Ahead Forecast Forecast ({update_h} h update)')
f.show()

err.T

,2018-12-10,2018-12-11,2018-12-12,2018-12-13,2018-12-14,2018-12-15,2018-12-16
Forecast nMAE %,6.5,5.8,4.9,3.7,9.9,1.2,2.0
Forecast nRMSE %,9.1,7.9,6.9,5.2,16.7,1.8,3.0
Persist nMAE %,4.3,4.1,4.1,3.6,13.9,2.0,0.9
Persist nRMSE %,7.2,6.8,6.6,5.6,23.9,3.2,2.0
Skill nMAE %,-50.9,-41.0,-20.4,-1.9,28.8,38.4,-113.0
Skill nRMSE %,-27.3,-16.0,-4.9,7.0,30.2,43.9,-46.8
